In [2]:
from google.colab import drive
drive.mount('cnontent/')

Drive already mounted at cnontent/; to attempt to forcibly remount, call drive.mount("cnontent/", force_remount=True).
Drive already mounted at cnontent/; to attempt to forcibly remount, call drive.mount("cnontent/", force_remount=True).


In [2]:
cd /content/cnontent/MyDrive/SSU/nural_nework/project

/content/cnontent/MyDrive/SSU/nural_nework/project


In [3]:
#import
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization, Dropout
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [4]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(len(gpus))
# if gpus:
#   try:
#     tf.config.experimental.set_memory_growth(gpus[0], True)
#     tf.config.experimental.set_memory_growth(gpus[1], True)

#   except RuntimeError as e:
#     # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
#     print(e)

In [4]:
#data preparing
def convert_data_csv_to_numpy(data, sorting = False): # pandas 통해 읽은 csv data numpy 형태로 변경
  if sorting == True:
    data = data.sort_values(by=[0], axis=0)

  label = np.array(data[0]) # csv file 에서 0번째 colum은 index임
  only_data = np.array(data.drop([0], axis = 1)) # csv file에서 0번 째 colum 탈락 -> data만 남게 됨
  # only_data = tf.convert_to_tensor(only_data, dtype=tf.float32)
  only_data = tf.keras.utils.normalize(only_data, axis=-1, order=2).reshape((-1, 28, 28, 1)) #data nomalize & reshaping
  return only_data, label

In [5]:
# #load
# train_df = pd.read_csv("./emnist-byclass-train.csv", header=None)
# test_df = pd.read_csv("./emnist-byclass-test.csv", header=None)

train_csv, test_csv = [], []
train_csv = pd.read_csv("./emnist-byclass-train.csv", header=None)

In [6]:
# #change to numpy
# X_train, y_train  = convert_data_csv_to_numpy(train_df)
# X_test, y_test  = convert_data_csv_to_numpy(test_df)

# train_df = None
# test_df = None

# train, train_label = convert_data_csv_to_numpy(train_csv, sorting = True)
X_train, y_train = convert_data_csv_to_numpy(train_csv, sorting = False)
train_csv = []

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [8]:
# print(X_train.shape, X_val.shape, X_test.shape)
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

(558345, 28, 28, 1) (139587, 28, 28, 1)
(558345,) (139587,)


In [9]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
# y_test = tf.keras.utils.to_categorical(y_test)

In [10]:
# print(y_train.shape,y_val.shape, y_test.shape)
print(y_train.shape,y_val.shape)

(558345, 62) (139587, 62)


In [11]:
X_train = tf.convert_to_tensor(X_train.copy(), dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val.copy(), dtype=tf.float32)

In [12]:
print(type(X_train))
print(type(X_val))
print(X_train.shape)
print(X_val.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
(558345, 28, 28, 1)
(139587, 28, 28, 1)


In [13]:
f_train_x = tf.image.grayscale_to_rgb(X_train)
f_val_x = tf.image.grayscale_to_rgb(X_val)

In [ ]:
# #final data split
# boundary = int(X_train.shape[0]*0.8)

# final_train = f_train_x[:boundary]
# final_train_y = y_train[:boundary]

# final_val = f_train_x[boundary:]
# final_val_y = y_train[boundary:]

In [14]:
f_train_x[0].shape

TensorShape([28, 28, 3])

In [15]:
mean = np.mean(X_train)
std = np.std(X_train)

In [16]:
print(mean)
print(std)

0.016407458
0.031722296


In [17]:
#efficientent _model0
IMG_SHAPE = (224, 224, 3)
model0 = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE, include_top=False, weights="imagenet", classifier_activation="softmax")

model0.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [23]:
#make new model _eff_model
tf.keras.utils.plot_model(model0) # to draw and visualize

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(62, activation = 'softmax')

eff_model = keras.models.Sequential()
# eff_model.add(tf.keras.layers.Normalization( axis=-1, mean=0.016425587, variance=0.03171295, input_shape = (28, 28, 3)))
eff_model.add(tf.keras.layers.Resizing(height = 224, width = 224, input_shape = (28, 28, 3)))
eff_model.add(model0)
eff_model.add(global_average_layer)
eff_model.add(tf.keras.layers.Dense(1000))
eff_model.add(tf.keras.layers.ReLU())
eff_model.add(prediction_layer)


eff_model.summary() # to see the list of layers and parameters


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 1000)              1281000   
                                                                 
 re_lu_1 (ReLU)              (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 62)                62062     
                                                      

In [19]:
print(f_train_x.shape)
print(y_train.shape)
# print(final_train_y.shape)
print(f_val_x.shape)
print(y_val.shape)
# print(final_val_y.shape)

(558345, 28, 28, 3)
(558345, 62)
(139587, 28, 28, 3)
(139587, 62)


In [24]:
eff_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)
batch_size = 8
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

eff_model_history = eff_model.fit(aug.flow(f_train_x, y_train, batch_size = batch_size), batch_size = 8, epochs=10, validation_data=(f_val_x, y_val), callbacks=[early_stopping_cb])

Epoch 1/10
69794/69794 [==============================] - 5359s 77ms/step - loss: 0.5281 - accuracy: 0.8163 - val_loss: 12.9393 - val_accuracy: 0.0443
Epoch 2/10
69794/69794 [==============================] - 5247s 75ms/step - loss: 0.4849 - accuracy: 0.8300 - val_loss: 15.8719 - val_accuracy: 0.0373
Epoch 3/10
69794/69794 [==============================] - 5259s 75ms/step - loss: 0.4744 - accuracy: 0.8339 - val_loss: 10.2276 - val_accuracy: 0.0419
Epoch 4/10
 9149/69794 [==>...........................] - ETA: 1:12:39 - loss: 0.4679 - accuracy: 0.8361